# Inverse-Model
Implementation steps
- [x] implement hessian-vector product
    - gradient convergence according to Taylor-ish test.
    - hessian approximation is close to finite-differences.
- [x] implement projected newton solver
    - still not tested with optimal values along boundary or outside bounds.
- [x] ensure correctness with measurement times not conforming to grid
    - measurements according to piecewise constant. Linear interpolation would require more work.
- [x] verify functioning model with linear interpolation of boundary concentrations
    - Does at least converge to a solution in the 2D case.
- [ ] extend to full 3d brain with proper data
- [ ] performance optimization by caching
- [ ] run full model for all subjects
- [ ] enable linear interpolation measurements

In [1]:
import numpy as np
import dolfin as df
import tqdm
import pint
from dolfin import inner, grad, dot

from glymphopt.datageneration import BoundaryConcentration
from glymphopt.finite_differences import gradient_finite_differences, hessian_finite_differences
from glymphopt.interpolation import LinearDataInterpolator
from glymphopt.io import read_mesh, read_function_data
from glymphopt.measure import measure
from glymphopt.minimize import projected_newton_solver
from glymphopt.operators import (
    mass_matrix, boundary_mass_matrix, stiffness_matrix, matrix_operator, bilinear_operator, matmul, zero_vector
)
from glymphopt.parameters import parameters_2d_default
from glymphopt.timestepper import TimeStepper

[garrus:86747] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.garrus.1000/jf.0/1891041280/shared_mem_cuda_pool.garrus could be created.
[garrus:86747] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


In [15]:
def vector2coeff(x: np.ndarray, *args):
#     return parameters_2d_default() | {key: xi for key, xi in zip(args, x)}
    T = pint.Quantity("1s")
    X = pint.Quantity("1mm")
    timescale = float(pint.Quantity("1s") / T)
    params = singlecomp_parameters(get_dimless_parameters(T, X))
    coefficients = remove_units(params)
    return remove_units(params) | {key: xi for key, xi in zip(args, x)}


def coeff2vector(coefficients: dict[str, float], *args):
    return np.array([coefficients[key] for key in args])

class Model:
    def __init__(self, V, D=None, g=None):
        D = D or df.Identity(V.mesh().topology().dim())
        
        domain = V.mesh()
        dx = df.Measure("dx", domain)
        ds = df.Measure("ds", domain)
        
        u, v = df.TrialFunction(V), df.TestFunction(V)
        self.M = df.assemble(inner(u, v) * dx)
        self.DK = df.assemble(inner(D * grad(u), grad(v)) * dx)
        self.S = df.assemble(inner(u, v) * ds)
        self.g = g or BoundaryConcentration(V)


def gradient_sensitivities(F, x, **kwargs):
    return np.array([F(x, ei, **kwargs) for ei in np.eye(len(x))])

def measure_interval(n: int, td: np.ndarray, timestepper: TimeStepper):
    bins = np.digitize(td, timestepper.vector(), right=True)
    return list(np.where(n == bins)[0])

class InverseProblem:
    def __init__(self, data_path, dt=0.1, g=None, D=None):
        domain = read_mesh(data_path)
        self.td, self.Yd = read_function_data(data_path, domain, "concentration")
        t_start = self.td[0]
        N = int(np.ceil(np.round((self.td[-1] - t_start) / dt, 12)))
        t_end = N * dt
        self.timestepper = TimeStepper(dt, (t_start, t_end))
        self.V = self.Yd[0].function_space()
        
        g = g or BoundaryConcentration(self.V)
        self.model = Model(self.V, g=g, D=D)
        
    def F(self, x):
        Y = self.forward(x)
        Ym = measure(self.timestepper, Y, self.td)
        _M_ = bilinear_operator(self.model.M)
        J = 0.5 * sum([
            _M_(Ym_i.vector() - Yd_i.vector(), Ym_i.vector() - Yd_i.vector()) 
            for Ym_i, Yd_i in zip(Ym, self.Yd)
        ])
        return J
    
    def gradF(self, x):
        dt = self.timestepper.dt
        model = self.model
        Y = self.forward(x)
        Ym = measure(self.timestepper, Y, self.td)
        P = self.adjoint(x, self.timestepper, Ym)
        G = [matmul(model.S, model.g(t)) for t in self.timestepper.vector()]
        _M_ = bilinear_operator(model.M)
        _DK_ = bilinear_operator(model.DK)
        _S_ = bilinear_operator(model.S)
        return dt * sum(
            np.array([
                _DK_(p.vector(), y.vector()),
                _M_(p.vector(), y.vector()),
                _S_(p.vector(), y.vector()) - p.vector().inner(g),
            ])
            for y, p, g in zip(Y[1:], P[:-1], G[1:])
        )

    def dF(self, x, dx):
        timestepper = self.timestepper
        coefficients = vector2coeff(x)
        Y = self.forward(x)
        dY = self.sensitivity(x, dx, Y)
        
        Ym = measure(self.timestepper, Y, self.td)
        dYm = measure(timestepper, dY, self.td)
        _M_ = bilinear_operator(self.model.M)
        return sum([_M_(ym.vector() - yd.vector(), dy.vector()) for ym, yd, dy in zip(Ym, self.Yd, dYm)])
    
    def hess(self, x):
        return np.array([
            self.hessp(x, ei) for ei in np.eye(len(x))
        ])
    
    def hessp(self, x, dx):
        dt = self.timestepper.dt
        Y = self.forward(x)
        Ym = measure(self.timestepper, Y, self.td)
        dY = self.sensitivity(x, dx, Y)
        dYm = measure(self.timestepper, dY, self.td)
        
        P = self.adjoint(x, self.timestepper, Ym)
        dP = self.second_order_adjoint(x, dx, dYm, P)
        
        model = self.model
        G = [matmul(model.S, model.g(t)) for t in self.timestepper.vector()]
        _DK_ = bilinear_operator(model.DK)
        _M_ = bilinear_operator(model.M)
        _S_ = bilinear_operator(model.S)
        return dt * sum(
            np.array([
                _DK_(dp.vector(), y.vector()) + _DK_(p.vector(), dy.vector()),
                _M_(dp.vector(), y.vector()) + _M_(p.vector(), dy.vector()),
                _S_(dp.vector(), y.vector()) - dp.vector().inner(g) + _S_(p.vector(), dy.vector()),
            ])
            for y, dy, p, dp, g in zip(Y[1:], dY[1:], P[:-1], dP[:-1], G[1:])
        )

    def forward(self, x):
        coefficients = vector2coeff(x, "a", "r", "k")
        a = coefficients["a"]
        r = coefficients["r"]
        k = coefficients["k"]
        
        timestepper = self.timestepper
        dt = timestepper.dt
        timepoints = timestepper.vector()
        Y = [
            df.Function(self.V, name="state") 
            for _ in range(len(timepoints))
        ]
        Y[0].assign(self.Yd[0])
        model = self.model
        M = model.M
        L = a * model.DK + r * model.M + k * model.S
        solver = df.LUSolver(M + dt * L)
        Mdot = matrix_operator(M)
        G = [matmul(model.S, model.g(t)) for t in timepoints]
#         for n in range(self.timestepper.num_intervals()):
        for n in range(self.timestepper.num_intervals()):
            solver.solve(Y[n+1].vector(), Mdot(Y[n].vector()) + dt * k * G[n+1])
        return Y
    
    def adjoint(self, x, timestepper, Ym) -> list[df.Function]:
        # TODO: Needs speedup. Should not allocate zero_vector at each iteration.
        coefficients = vector2coeff(x, "a", "r", "k")
        a = coefficients["a"]
        r = coefficients["r"]
        k = coefficients["k"]
        
        
        dt = timestepper.dt
        timepoints = timestepper.vector()
        
        model = self.model
        M = model.M
        L = a * model.DK + r * model.M + k * model.S
        solver = df.LUSolver(M + dt * L)

        P = [
            df.Function(self.V, name="adjoint") 
            for _ in range(len(timepoints))
        ]
        Mdot = matrix_operator(M)
        num_intervals = timestepper.num_intervals()
        for n in range(num_intervals, 0, -1):
            nj = measure_interval(n, self.td, self.timestepper)
            jump = sum((matmul(M, (Ym[j].vector() - Yd[j].vector())) for j in nj))#, start=zero_vector(self.V))

            solver.solve(
                P[n-1].vector(),
                Mdot(P[n].vector()) - jump,
            )
        return P
    
    def sensitivity(self, x, dx, Y) -> list[df.Function]:
        coefficients = vector2coeff(x, "a", "r", "k")
        a = coefficients["a"]
        r = coefficients["r"]
        k = coefficients["k"]
        dD, dr, dk = dx

        timestepper = self.timestepper
        dt = timestepper.dt
        timepoints = timestepper.vector()
        dY = [df.Function(self.V, name="sensitivity") for _ in range(len(timepoints))]

        model = self.model
        M = model.M
        L = a * model.DK + r * model.M + k * model.S
        solver = df.LUSolver(M + dt * L)
        
        dL = dD * model.DK + dr * model.M + dk * model.S
        Mdot = matrix_operator(M)
        dLdot = matrix_operator(dL)
        Sdot = matrix_operator(model.S)
        G = [matmul(model.S, model.g(t)) for t in timepoints]
        for n, t in enumerate(timepoints[1:], start=0):
            solver.solve(
                dY[n+1].vector(),
                Mdot(dY[n].vector()) - dt * dLdot(Y[n+1].vector()) + dt * dk * G[n+1]
            )
        return dY
    
    def second_order_adjoint(self, x, dx, dYm, P):
        coefficients = vector2coeff(x, "a", "r", "k")
        a = coefficients["a"]
        r = coefficients["r"]
        k = coefficients["k"]
        dD, dr, dk = dx

        timestepper = self.timestepper
        dt = timestepper.dt
        timepoints = timestepper.vector()
        
        dP = [df.Function(self.V, name="second-order-adjoint") for _ in range(len(timepoints))]

        model = self.model
        M = model.M
        L = a * model.DK + r * model.M + k * model.S
        dL = dD * model.DK + dr * model.M + dk * model.S
        solver = df.LUSolver(M + dt * L)

        Mdot = matrix_operator(M)
        dLdot = matrix_operator(dL)
        num_intervals = timestepper.num_intervals()
        for n in range(num_intervals, 0, -1):
            nj = measure_interval(n, self.td, self.timestepper)
            jump = sum((matmul(M, dYm[j].vector()) for j in nj))
            solver.solve(
                dP[n - 1].vector(),
                Mdot(dP[n].vector()) - dt * dLdot(P[n-1].vector()) - jump,
            )
        return dP

In [16]:
from glymphopt.parameters import singlecomp_parameters, get_dimless_parameters, remove_units

In [17]:
coefficients

{'a': 0.13354545454545455,
 'r': 9.090909090909091e-06,
 'k': 0.0017272727272727275,
 'phi': 0.22,
 'phi_sas': 0.8}

In [18]:
data_path = "../resources/concentrations.hdf"
data_path = "/home/jorgen/gonzo/mri_processed_data/sub-01/modeling/resolution32/data.hdf"
domain = read_mesh(data_path)
td, Yd = read_function_data(data_path, domain, "concentration")
td, Y_bdry = read_function_data(data_path, domain, "boundary_concentration")
g = LinearDataInterpolator(td, Y_bdry)
D = 1.3e-4 * df.Identity(domain.topology().dim())

T = pint.Quantity("1s")
X = pint.Quantity("1mm")
timescale = float(pint.Quantity("1s") / T)
params = singlecomp_parameters(get_dimless_parameters(T, X))
coefficients = remove_units(params)


# coefficients = parameters_2d_default()
x0 = coeff2vector(coefficients, "a", "r", "k")
x1 = 0.9 * x0


problem = InverseProblem(data_path, dt=3600, g=g, D=D)
# problem.F(x0)

Calling FFC just-in-time (JIT) compiler, this may take some time.


array([1.20190909e-01, 8.18181818e-06, 1.55454545e-03])

# Convergence test for gradient

In [5]:
import matplotlib.pyplot as plt
gradF = problem.gradF(x1)

hs = [0.5**i * 1e-1 for i in range(8)]
errors = np.nan * np.zeros_like(hs)
for i, h in enumerate(hs):
    dF_findiff = gradient_finite_differences(problem.F, x1, h=h)
    errors[i] = np.linalg.norm(gradF - dF_findiff)
    if i > 0:
        print(f"h = {h}, errornorm = {errors[i]}, rel={errors[i-1]/errors[i]}")
    else:
        print(f"h = {h}, errornorm = {errors[i]}, rel=")

plt.loglog(hs, errors, "x-")
plt.show()

h = 0.1, errornorm = 2.1129553255916287, rel=
h = 0.05, errornorm = 0.524997648791058, rel=4.024694835219265
h = 0.025, errornorm = 0.13104837305511105, rel=4.006136333873261


KeyboardInterrupt: 

# Hessians

In [ ]:
H = problem.hess(x1)
H

In [ ]:
H_fd = hessian_finite_differences(problem.F, x1, h=1e-5)
H_fd

In [ ]:
(H_fd - H) / H

In [7]:
x1

array([9.090e+00, 9.000e-04, 1.179e+00])

In [ ]:
import time as pytime
import scipy.optimize
bounds = scipy.optimize.Bounds([1e-1, 0, 0], [np.inf, np.inf, np.inf])
x0 = [2.0, 1e-8, 1.5e-3]
sol_x = projected_newton_solver(
    problem.F,
    problem.gradF,
    problem.hessp,
    bounds=bounds,
    x0=x0
)
toc = pytime.time()
sol_x

0	[2.00000399e+00 6.31493247e-06 2.74407363e-03]	5242.773181692028	[ 8.03932689e+02 -3.78809260e+08  2.05352212e+05]	1.0
1	[2.00001766e+00 1.56350568e-05 4.46497052e-03]	3821.9678487129822	[ 8.74214642e+02 -1.67092664e+08  8.69271104e+04]	1.0
2	[2.00005416e+00 2.93071085e-05 6.91982628e-03]	2884.8812546967897	[ 7.76663021e+02 -7.36974484e+07  3.76825268e+04]	1.0
